In [1]:
# Configure the necessary Spark environment
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install -- currently using spark 2.4
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

#os.environ['PYSPARK_SUBMIT_ARGS']="--jars /work/ericr/spark/sparkdev/postgresql.jar --executor-memory 40g --executor-cores 16 pyspark-shell"

# Initialize PySpark to predefine the SparkContext variable 'sc'
#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [11]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import flatten, explode, col
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
import json



In [3]:
spark = SparkSession.builder.appName('Basic').config("spark.executor.extraJavaOptions","--executor-memory 40G --executor-cores 40 --driver-class-path $SPARK_HOME/postgresql.jar").getOrCreate()

In [4]:
spark.conf.set("spark.sql.caseSensitive","true")

In [5]:
dbconfig = {"url": "jdbc:postgresql://localhost/qxedb",
            "dbtable": "gateway_eventnotification",
            "user": "hermes",
            "password": "mysecret",
            "driver": "org.postgresql.Driver"}

In [6]:
df = spark.read.jdbc(url="jdbc:postgresql://localhost/qxedb", 
                      table="gateway_eventnotification",
                      properties=dbconfig)

In [7]:
sqlcontext = SQLContext(spark.sparkContext)

In [8]:
payload_df = sqlcontext.read.json(df.rdd.map(lambda r: r.payload))

In [12]:
payload_df.printSchema()

root
 |-- data_type: string (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- eventCategory: string (nullable = true)
 |    |    |-- eventDetail: struct (nullable = true)
 |    |    |    |-- brand: string (nullable = true)
 |    |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |   

In [20]:
df2=payload_df.drop("onDurationTime (sec)")

In [22]:
df2.printSchema()

root
 |-- data_type: string (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- eventCategory: string (nullable = true)
 |    |    |-- eventDetail: struct (nullable = true)
 |    |    |    |-- brand: string (nullable = true)
 |    |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |    |-- addr: string (nullable = true)
 |   

In [13]:
payload_df = payload_df.withColumn("index",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [14]:
x = payload_df.select(["index",explode(payload_df.events).alias("event"),
                       "henq_metadata",
                       "links",
                       "metadata",
                       "originator",
                       "payloadManifestJec",
                       "subscriptionId",
                       "system_check_id",
                       "version"])

In [ ]:
#x2 = x.select(["index",
#                       "event",
#                       "henq_metadata",
#                       explode(payload_df.links).alias("link"),
#                       "metadata",
#                       "originator",
#                       "payloadManifestJec",
#                       "subscriptionId",
#                       "system_check_id",
#                       "version"])

In [15]:
x=x.na.drop(subset=["event.sequenceNumber"])
x=x.na.drop(subset=["event.eventDetail.serialNumber"])

In [16]:
x.printSchema()

root
 |-- index: integer (nullable = true)
 |-- event: struct (nullable = true)
 |    |-- dateTime: string (nullable = true)
 |    |-- eventCategory: string (nullable = true)
 |    |-- eventDetail: struct (nullable = true)
 |    |    |-- brand: string (nullable = true)
 |    |    |-- connectionInfo: struct (nullable = true)
 |    |    |    |-- localIpv4Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |-- sourceInSameSubnet: string (nullable = true)
 |    |    |    |-- sourceIpv4Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |    |-- subnetMask: string (nullable = true)
 |    |    |    |-- sourceIpv6Address: struct (nullable = true)
 |    |    |    |    |-- addr: string (nullable = true)
 |    |    |    |-- sourceMacAddress: string (nullable = true)
 |    |    |-- connectionState: string (nullable = true)
 |    

In [17]:
x.show(truncate=18)

+-----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-------+
|index|             event|     henq_metadata|             links|          metadata|        originator|payloadManifestJec|    subscriptionId|system_check_id|version|
+-----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-------+
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  410|[2018-03-15T10:...|[2018-03-15 20:...|[[/cdm/telemetr...|              null|              null|              null|f2dece9d-3087-4...|           null|  1.0.0|
|  668|[20

In [18]:
#x=x.withColumnRenamed("onDurationTime (sec)","onDurationTime")
x.write.parquet("/work/ericr/payload.parquet",compression="gzip",mode="overwrite")


Py4JJavaError: An error occurred while calling o93.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.

In [ ]:
dfp = spark.read.parquet("/work/ericr/payload.parquet")

In [ ]:
dfp.printSchema()

In [ ]:
dfp.orderBy("sequenceNumber",ascending=False).show(truncate=18)

In [ ]:
drec = dfp.groupBy("sequenceNumber").count()

In [ ]:
drec.count()

In [ ]:
d=dfp.select("eventCategory")

In [ ]:
d.show()

In [ ]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

In [ ]:
dr = dfp.withColumn("index",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [ ]:
dr.show(truncate=18)